# Challenge 01 - Prompt Engineering

## 1. Parameter Experimentation
Let's first set up the Challenge. Load the API key and relevant Python libraries using the cells below.

### 1.1 Preparation
#### 1.1.1 Environment Setup


#### Create or Update the environment file ".env"

#### Create the environment file ".env"
Create an environment file named ".env" under the same folder with your notebooks. We've included a sample environment file for your reference. (OpenAIFundamentals/Student/Resources/.env-sample). Please feel free to make any modification as needed.

If using a provided Codespace, the environment file will be found under the data and notebooks folder.

**NOTE:** Make sure to keep the name of each model the same as the model deployment name you set in Azure OpenAI.

**NOTE:** Please be aware that some AOAI models are deprecated. You should create the ones available as needed instead. To complete this Challenge, gpt-35-turbo is the only model you need.

For information on current models, see: [Azure OpenAI Service models](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)

If you are using Azure ML Studio, if you want to change the model name in the environment file, please run the following code to make a change

In [ ]:
# os.environ[{MODEL YOU WANT TO CHANGE}] = {NEW MODEL NAME}
# print({MODEL NAME})

####  Set up Azure OpenAI Environment

In [ ]:
import openai
import os
import json
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

In [ ]:
API_KEY = os.getenv("OPENAI_API_KEY")
assert API_KEY, "ERROR: Azure OpenAI Key is missing"
openai.api_key = API_KEY

RESOURCE_ENDPOINT = os.getenv("OPENAI_API_BASE","").strip()
assert RESOURCE_ENDPOINT, "ERROR: Azure OpenAI Endpoint is missing"
assert "openai.azure.com" in RESOURCE_ENDPOINT.lower(), "ERROR: Azure OpenAI Endpoint should be in the form: \n\n\t<your unique endpoint identifier>.openai.azure.com"

openai.api_base = RESOURCE_ENDPOINT
openai.api_type = os.getenv("OPENAI_API_TYPE")
openai.api_version = os.getenv("OPENAI_API_VERSION")

chat_model=os.getenv("CHAT_MODEL_NAME")


#### 1.1.2 Parameter Explanation
We specify the hyperparameters for the Azure OpenAI models within the helper functions. Users can tune the parameters according to different needs.


###### **Temperature**
Temperature ranges from 0 to 2.
Here is a quick breakdown of how it works:
- Low temperature (0 to 0.3): More focused, coherent, and conservative outputs.
- Medium temperature (0.3 to 0.7): Balanced creativity and coherence.
- High temperature (> 0.7): Highly creative and diverse, but potentially less coherent.

###### **Top_p**
Sets the probability mass cutoff for token sampling, affecting the breadth of options the AI considers. 
Higher values lead to more randomness, while lower values result in more focused outputs.

The “top_p” parameter is like a filter that controls how many different words or phrases the language model considers when it’s trying to predict the next word. If you set the “top p” value to 0.5, the language model will only consider the 50 most likely words or phrases that might come next. But if you set the “top p” value to 0.9, the language model will consider the 90 most likely words or phrases.

And as "top_p" and "temperature" performs similar job as hyperparameter, we usually only tune one of them instead of both.

###### **Max_tokens**
Max_tokens determine the maximum length of the generated text. By setting a limit, you can control how much text the LLM model will return, making sure it doesn't give too long of an answer.

###### **Frequency_penalty**
Frequency_penalty makes sure that the text that is generated is varied by giving a penalty to tokens that have already been used in the response.

It ranges from -2.0 to 2.0, with higher values resulting in more diverse output.

Positive values penalize new tokens based on their existing frequency in the text so far, decreasing the model's likelihood to repeat the same line verbatim.

#### 1.1.3 Helper Function
Throughout this hack, we will use OpenAI's `gpt-3.5-turbo` model and the [chat completions endpoint](https://platform.openai.com/docs/guides/chat). 

This helper function will make it easier to use prompts and look at the generated outputs.

**get_chat_completion** helps create the OpenAI response using the chat model of your choice.

**get_completion_from_messages** helps create the OpenAI response using the chat model of your choice, enabling chat history.


In [ ]:
def get_chat_completion(prompt, model=chat_model):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=0, # this is the degree of randomness of the model's output
        max_tokens = 200,
        top_p = 1.0
    )
    return response.choices[0].message["content"]

In [ ]:
def get_completion_from_messages(messages, model=chat_model, temperature=0):
    response = openai.ChatCompletion.create(
        engine=model,
        messages=messages,
        temperature=temperature # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]



#### Try out helper functions

In [ ]:
messages =  f"""
tell me a joke.
"""
response = get_chat_completion(messages)
print(response)

In [ ]:
messages =  [      
    {'role':'user', 'content':'tell me a joke'},   
    {'role':'assistant', 'content':'Why did the chicken cross the road'},   
    {'role':'user', 'content':'I don\'t know'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

#### Student Tasks (open questions):
1. Create a completion function for a higher temperature between (0,1).
2. Create a completion function for a lower max_token for shorter response.
3. Create acompletion functions with 2 different diversity penalty values between (0,2).

Try out the completion functions you create on the previous case, compare the result you get.

In [ ]:
# Try out a completion function for a higher temperature between (0,1)

In [ ]:
# Try out a completion function for a lower max_token for shorter response

In [ ]:
# Try out completion functions with 2 different diversity penalty values between (0,2)

### 1.2 System Message Engineering
Users can achieve the response from models in their desired tone through adjusting the system message.

#### 1.2.1 Change of Tone

In [ ]:
messages =  [      
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

In [ ]:
messages =  [  
    {'role':'system', 'content':'You are an assistant that speaks like Shakespeare.'},    
    {'role':'assistant', 'content':'How can I help you?'},
    {'role':'user', 'content':'tell me a joke'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

#### Student Task (open question):
Make the assistant tell the joke in the tone of your favorite character by editing the system message.


In [ ]:
# Make the assistant tell the joke in the tone of your favorite character by editing the system message.


#### 1.2.2 Remind the Company Name

In [ ]:
messages =  [  
{'role':'system', 'content':'You are friendly chatbot.'},  
{'role':'user', 'content':'Hi, my name is Mel.'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},  
{'role':'user', 'content':'Yes, can you remind me which company I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

#### Student Task: 
Make the assistant give a reliable reminder of company name by giving context in the system message.

In [ ]:
# Make the assistant give a reliable reminder of company name by giving context in system message.

messages =  [  
{'role':'system', 'content':'You are friendly chatbot.Taking to people work for Microsoft.'},
{'role':'user', 'content':'Hi, my name is Melody'},
{'role':'assistant', 'content': "Hi! It's nice to meet you. \
Is there anything I can help you with today?"},
{'role':'user', 'content':'Yes, you can remind me, Which company do I work for?'}  ]
response = get_completion_from_messages(messages, temperature=1)
print(response)

## 2. Iterative Prompting Principles
It is generally a good practice to perform iterative prompting so the model could generate the most appropriate response given the user's specification. 
- **Principle 1: Write clear and specific instructions**
- **Principle 2: Give the model time to “think”**



### 2.1 Write clear and specific instructions

#### Tactic 1: Delimiters

#### Student Task:
Use delimiters to clearly indicate distinct parts of the input

Delimiters can be anything like: ````, """, < >, `<tag> </tag>`, `:`

In [ ]:
text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

In [ ]:
# Use delimiters to clearly indicate distinct parts of the input, and ask the model to summarize the text.


prompt = f"""
give the text delimited by triple backticks a newspaper title.
Text:
```{text}```
"""
response = get_chat_completion(prompt)

print(response)

#### Tactic 2: Summarization: specify word counts, extract information

#### Text to summarize

In [ ]:
game_review = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

#### Student Task:
Summarize with a focus on the special parts of the 2020 Summer Olympics and with a word limit

In [ ]:
# Summarize with a focus on the special parts of the 2020 Summer Olympics and with a word limit

prompt = f"""
Your task is to generate a short summary of the 2020 Summer Olympics report to show how important \
this Olympics event means to Japan. 

Summarize the review below, delimited by triple 
backticks, in at most 50 words, and focusing on any aspects that shows the differences of this event. 

Review: ```{game_review}```
"""

response = get_chat_completion(prompt)
print(response)

#### Student Task:
Try "extract" instead of "summarize"

In [ ]:
# Try "extract" instead of "summarize"

prompt = f"""
Your task is to extract relavant information from the 2020 Summer Olympics report to show how important \
this Olympics event means to Japan. 

From the review below, delimited by triple 
backticks, extract the information focusing on any aspects that shows the differences of this event. \
Limit to 50 words.

Review: ```{game_review}```
"""

response = get_chat_completion(prompt)
print(response)

#### Tactic 3: Inferring: ask for emotions, sentiment, or topics 

#### Student Task:
Identify types of emotions and sentiment (positive/negative) of the review below

Format in a JSON object

In [ ]:
review = """
Philip Barker of Inside the Games opined that for many athletes and supporters, \
the tone of the ceremony was dignified and appropriate. Hashimoto stated in a press interview that the flame would "quietly go out", \
which he felt that "It was an apt description of a dignified and low key Ceremony which conveyed a sense of gratitude that the Games had been able to take place at all."\
Dominic Patten of Deadline Hollywood argued that the ceremony was an "uneven mixtape" of contrasts, \
comparing the low-key "celebration of the culture of the Asian power and brow moping acknowledgement of the pandemic" to the jubilant Paris segment, \
as well as clichÃ©-filled speech of Thomas Bach. Alan Tyres of The Daily Telegraph discussed the IOC updated motto as a sign of things to come. \
He stated, "The updated Olympic motto of 'faster, higher, \
stronger â€“ together' fits with how sport is covered and contextualised at this moment in history: \
inclusion, diversity, justice and a duty of care to the athletes must be taken into consideration as much as performance." \
He also discussed the strangeness of the ceremony, as it was performed without a stadium audience.
"""

In [ ]:
# Identify types of emotions and sentiment (positive/negative) of the review above. Format in a JSON object.


prompt = f"""
Identify the following items from the review text: 
- a list of emotions that the writer of the following review is expressing
- Sentiment (positive or negative)

The review is delimited with triple backticks. \
Format your response as a JSON object with \
"Emotion" and "Sentiment" as the keys.
If the information isn't present, use "unknown" \
as the value.
Make your response as short as possible.
Format the Anger value as a boolean.

Review text: '''{review}'''
"""
response = get_chat_completion(prompt)
print(response)

#### Student Challenge:
Infer 3 topics of the story below

In [ ]:
story = """
The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, \
was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, \
with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, \
on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, \
the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). \
However, the event retained the Tokyo 2020 branding for marketing purposes. \
It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. \
The Games were the most expensive ever, with total spending of over $20 billion. \
The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), \
and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. \
The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, \
South Korea and preceding the 2022 Winter Olympics in Beijing, China. \
Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.
New events were introduced in existing sports, including 3x3 basketball, \
freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. \
New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. \
The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, \
sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program. \
The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). \
Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. \
Great Britain finished fourth, with a total of 22 gold and 64 medals. \
The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. \
Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.
"""

In [ ]:
# Infer 3 topics of the story above. 

prompt = f"""
Determine three topics that are being discussed in the \
following text, which is delimited by triple backticks.

Make each item one or two words long. 

Format your response as a list of items separated by commas.

Text sample: '''{story}'''
"""
response = get_chat_completion(prompt)
print(response)

#### Tactic 4: Transforming: specify target language and writing style, and ask for grammar check

#### Student Task:
Universal Translator

People all over the world want to know the Olympic game news in their native language. In this case, the news needs to be translated into different languages. Translate each news item below into both Korean and English.

In [ ]:
news = [
"Palestine received a universality invitation from FINA to send two top-ranked swimmers (one per gender) in their respective individual events to the Olympics, \
based on the FINA Points System of June 28, 2021.",
"比赛包括两轮：预赛和决赛。预赛成绩最好的8次接力队晋级决赛。必要时使用游泳比赛来打破平局以晋级下一轮。"]

In [ ]:
for n in news:
    prompt = f"Tell me what language this is: {n}"
    lang = get_chat_completion(prompt)
    print(f"Original message ({lang}): {n}")

    prompt = f"""
    Translate the following  text to English \
    and Korean: ```{n}```
    """
    response = get_chat_completion(prompt)
    print(response, "\n")

#### Student Task:
Tone Transformation

Writing can vary based on the intended audience. ChatGPT can produce different tones. Transform the following message into a business letter.

In [ ]:
trans_message = "David, it's John! OMG, the Olympic game is so crazy"

In [ ]:
prompt = f"""
Translate the following from slang to a business letter: 
{trans_message}
"""
response = get_chat_completion(prompt)
print(response)

#### Student Task:
Format Conversion

ChatGPT can translate between formats. The prompt should describe the input and output formats. Convert the following JSON data into HTML format.

In [ ]:
data_json = { "The 2020 Summer Olympics Opening Ceremony audience name list" :[ 
    {"name":"Shyam", "email":"shyamjaiswal@gmail.com"},
    {"name":"Bob", "email":"bob32@gmail.com"},
    {"name":"Jai", "email":"jai87@gmail.com"}
]}

In [ ]:
prompt = f"""
Translate the following python dictionary from JSON to an HTML \
table with column headers and title: {data_json}
"""
response = get_chat_completion(prompt)
print(response)

#### Student Task:
Spellcheck and Grammar check the following text. 

To signal to the LLM that you want it to proofread your text, you instruct the model to 'proofread' or 'proofread and correct'.

In [ ]:
check_text = [ 
  "Alongside the main Emblem blue, the five other colors use in the branding of the 2020 Games is : Kurenai red,   Ai blue,   Sakula pink,   Fuji purple, and   Matsuba green.",
  "The competition have three round: heats, semifinals, and a final."
]

In [ ]:
for t in check_text:
    prompt = f"""Proofread and correct the following text
    and rewrite the corrected version.:
    ```{t}```"""
    response = get_chat_completion(prompt)
    print(response)

#### Tactic 5: Expanding: customize the automated reply

#### Student Task:
Customize the automated reply to the following customer email.

The customer faced an issue while buying the Olympics game ticket.

In [ ]:
# given the sentiment from the tactic on "inferring",
# and the original customer message, customize the email
sentiment = "negative"

# review for a ticket transaction
review = f"""
I bought the ticket of "Men's 100 metre freestyle swimming" game last week.\
The transaction went through successfully. However, I still have not received the ticket.\
Over one week has passed.\
"""

In [ ]:
prompt = f"""
You are a customer service AI assistant.
Your task is to send an email reply to a valued customer.
Given the customer email delimited by ```, \
Generate a reply to thank the customer for their review.
If the sentiment is positive or neutral, thank them for \
their review.
If the sentiment is negative, apologize and suggest that \
they can reach out to customer service. 
Make sure to use specific details from the review.
Write in a concise and professional tone.
Sign the email as `AI customer agent`.
Customer review: ```{review}```
Review sentiment: {sentiment}
"""
response = get_chat_completion(prompt)
print(response)

#### Tactic 6: Chatbot: personalize conversations for specific tasks or behaviors

#### Student Task:
Create a conversation with chatbot to know where the 2020 Summer Olympics is held.

In [ ]:
messages =  [      
    {'role':'user', 'content':'tell me about the 2020 Summer Olympics'},   
    {'role':'assistant', 'content':'What do you want to know?'},   
    {'role':'user', 'content':'Where is the 2020 Summer Olympics held?'}
]
response = get_completion_from_messages(messages, temperature=1)
print(response)

### 2.2 Give the model time to “think” 

#### Tactic 1: Specify the steps required to complete a task
Sometimes you can help the model "slow down" and give more robust, detailed answers by specifying the steps it should take.

Let's ask for output in multiple specified formats.

In [ ]:
prompt = f"""
Your task is to help a journalist summarize information from the article for publication.

Write a title based on the information provided in the context delimited by triple backticks. 
The title should be short, catchy, and reflective of the article's narrative.

After the title, generate five keywords from the context.

After the keywords, include a table to organize the information. 
The table should have two columns. In the first column is the title.
In the second column include the keywords as a list.

Give the table the title 'Article Publishing Information'.

Format everything as HTML that can be used in a website.
Place the title in a <div> element.

Context: ```{text}

""" 


get_chat_completion(prompt)

#### Tactic 2: Instruct the model to work out its own solution

There are two main methods we will demonstrate in this section to get the model to work through a problem instead of rushing to a conclusion, chain-of-thought and chaining. The strategies can lead to increased accuracy, detail, and the ability to work through complex challenges.


2.1 - Chain-of-thought prompting
- Ask the model to reason
- One-shot example
- Chatbot reasoning

2.2 - Chaining

Let's continue working with the Olympics dataset.

#### 2.2.1 Chain-of-Thought Prompting

Let's do a bit of math. GPT models occasionally don't do super well on solving direct math problems, so let's walk GPT through the problem.

Let's break down tasks into smaller pieces.

Read more about methods and whitepaper research here: https://github.com/openai/openai-cookbook/blob/main/techniques_to_improve_reliability.md

#### Tactic 1: You can start by specifically asking the model to simply think step-by-step.

In [ ]:

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

# From Azure documentation
prompt = "Who was the most decorated (maximum medals) individual athlete in the Olympic games that were held at Sydney? Take a step-by-step approach in your response, cite sources and give reasoning before sharing final answer in the below format: ANSWER is: <name>"
get_chat_completion(prompt)




In [ ]:
# Another example
prompt = f"""
What is the largest time zone difference is between the top two countries who 
won the most gold medals in the 2020 Tokyo olympics?

Use the context below and think aloud as you solve the problem, step-by-step.

Context: {text}
"""
get_chat_completion(prompt)

#### Tactic 2: One-shot example 
Another common tactic is to provide one example of a query and an ideal response. The model will learn from that example and apply the patterns to a new question.

In [ ]:
# Notice how this response may not be ideal, or the most accurate.
prompt = f"""
    The United States has 113 total medals, 39 of which are gold medals. 
    
    Great Britain has 64 medals and 22 gold medals. 
    
    How many more silver and bronze medals does the United States have over Great Britain?
"""

get_chat_completion(prompt)

In [ ]:
# Give the model a one-shot example to solve the question more thoroughly

prompt = f"""
Question: The United States has 113 total medals, 39 of which are gold medals. How many medals are silver or bronze?
Answer:
[Step 1] - There are three types of medals: gold, silver, and bronze
[Step 2] - We know the gold and total counts of medals, so the number of silver and bronze medals is the difference between the gold (39) and total (113) counts. 
            113-39=74, so there are 74 silver and bronze medals combined. The answer is 74.

===

Answer the following question using similar steps above.

Question: China has 89 total medals, 38 of which are gold medals. How many silver and bronze medals do they have?
Answer:
"""

get_chat_completion(prompt)

#### Tactic 3: Chatbot chain-of-thought reasoning
You can build in logic using variables so the chatbot can dynamically demonstrate specific ways of thinking about a problem.

**The input place is at the top of the screen. You should see a popup. Type 'quit' if you want to exit.**

In [ ]:
# Ask the bot to help you make a decision such as deciding whether to take a job or choose between restaurants.
# If the model does not immediately respond to your query, wait 1-5 seconds and retype it.
# If it is not allowing you to give an input, restart the kernel in the navigation bar.
# Type "quit" to end the session

context = '''
        You are a decision bot. Your job is to help come to decision by asking a series of questions one at a time and coming to a reasonable decision based on the information provided.

        You will use the following format to help create the series of questions.

        Template: 
        [Problem/Scenario/Question]: [Provide a brief description of the problem, scenario, or question.]

        Chain of thought:

        [Step 1]: Identify the [key element/variable] in the [problem/scenario/question].
        [Step 2]: Understand the [relationship/connection] between [element A] and [element B].
        [Step 3]: [Analyze/Evaluate/Consider] the [context/implication] of the [relationship/connection] between [element A] and [element B].
        [Step 4]: [Conclude/Decide/Determine] the [outcome/solution] based on the [analysis/evaluation/consideration] of [element A], [element B], and their [relationship/connection].
        [Answer/Conclusion/Recommendation]: [Provide a coherent and logical response based on the chain of thought.]

        You will guide the user though a series of questions one at a time. The first question is broad, and they subsequent questions become more specific. 

        Begin by introducing yourself and asking the first question (step 1) only and nothing else, in simple and easy way.
        '''

conversation=[{"role": "system", "content": context}]

while(True):
    print(len(conversation))
    if len(conversation) == 1:
        response = get_completion_from_messages(conversation)
        conversation.append({"role": "assistant", "content": response})
        print("\n" + response + "\n")
    
    user_input = input('Enter your response: ')
    if user_input.lower() == "quit":
        break 
    conversation.append({"role": "user", "content": user_input})
    
    response = get_completion_from_messages(conversation)
    conversation.append({"role": "assistant", "content": response})
    print("\n" + response + "\n")


#### 2.2.2 - Chaining
Similar to some earlier examples, you can use model outputs from previous queries into other queries. We will show you later in the Hack how to do this at scale.

In [ ]:
# Extract medal counts for each country from the news article
# Write an article about the summarized information
# Provide a title for the summary

text = f"""
    The 2020 Summer Olympics, officially the Games of the XXXII Olympiad and also known as Tokyo 2020, was an international multi-sport event held from 23 July to 8 August 2021 in Tokyo, Japan, with some preliminary events that began on 21 July 2021. Tokyo was selected as the host city during the 125th IOC Session in Buenos Aires, Argentina, on 7 September 2013.Originally scheduled to take place from 24 July to 9 August 2020, the event was postponed to 2021 on 24 March 2020 due to the global COVID-19 pandemic, the first such instance in the history of the Olympic Games (previous games had been cancelled but not rescheduled). However, the event retained the Tokyo 2020 branding for marketing purposes. It was largely held behind closed doors with no public spectators permitted due to the declaration of a state of emergency in the Greater Tokyo Area in response to the pandemic, the first and only Olympic Games to be held without official spectators. The Games were the most expensive ever, with total spending of over $20 billion.The Games were the fourth Olympic Games to be held in Japan, following the 1964 Summer Olympics (Tokyo), 1972 Winter Olympics (Sapporo), and 1998 Winter Olympics (Nagano). Tokyo became the first city in Asia to hold the Summer Olympic Games twice. The 2020 Games were the second of three consecutive Olympics to be held in East Asia, following the 2018 Winter Olympics in Pyeongchang, South Korea and preceding the 2022 Winter Olympics in Beijing, China. Due to the one-year postponement, Tokyo 2020 was the first and only Olympic Games to have been held in an odd-numbered year and the first Summer Olympics since 1900 to be held in a non-leap year.\nNew events were introduced in existing sports, including 3x3 basketball, freestyle BMX and mixed gender team events in a number of existing sports, as well as the return of madison cycling for men and an introduction of the same event for women. New IOC policies also allowed the host organizing committee to add new sports to the Olympic program for just one Games. The disciplines added by the Japanese Olympic Committee were baseball and softball, karate, sport climbing, surfing and skateboarding, the last four of which made their Olympic debuts, and the last three of which will remain on the Olympic program.The United States topped the medal count by both total golds (39) and total medals (113), with China finishing second by both respects (38 and 89). Host nation Japan finished third, setting a record for the most gold medals and total medals ever won by their delegation at an Olympic Games with 27 and 58. Great Britain finished fourth, with a total of 22 gold and 64 medals. The Russian delegation competing as the ROC finished fifth with 20 gold medals and third in the overall medal count, with 71 medals. Bermuda, the Philippines and Qatar won their first-ever Olympic gold medals. Burkina Faso, San Marino and Turkmenistan also won their first-ever Olympic medals.'
"""

prompt = f"""
    Based on the context below, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""
num_medals_dict = get_chat_completion(prompt)

prompt = f"""
    Write a brief article about the winners and losers of the Olympics based on medal count:
    {num_medals_dict}
"""
summary = get_chat_completion(prompt)
print(summary)

prompt = f"""
    Give the summary a title in 5 words:
    {summary}
"""
title = get_chat_completion(prompt)
print(title)

#### Student Task (Chaining): 

Your job is now to write code that will determine the country that won the most silver and bronze medals combined.

We can see that the model performs poorly on answering the question directly.

In [ ]:
prompt = f"""
    Based on the context, which country had the most silver and bronze medals?
    Context: {text}
"""

get_chat_completion(prompt)

Write prompts in the cell below that will help the model answer the question by breaking down the tasks into different steps.

You should be able to get the model to answer the question in 2-3 steps.

In [ ]:
### STUDENT TASK - WRITE CODE HERE ###



# Extract medal counts for each country from the news article and output a JSON object
prompt = f"""
    Based on the context, write a JSON object that contains the number of gold and total medals for each country.
    Context: {text}
"""
num_medals_dict = get_chat_completion(prompt)

# Add a new key representing the silver and bronze medals to the JSON object
prompt = f"""
    Calculate the number of silver and bronze (non-gold medals) based on the difference between gold and total medals, and add these counts to the JSON with the key "silver_bronze":
    {num_medals_dict}
"""
new_dict = get_chat_completion(prompt)

# Identify the country with the most silver and bronze medals
prompt = f"""
    Which country had the most silver and bronze medals?
    {new_dict}
"""
print(get_chat_completion(prompt))